# Daily data - Feature induction
The analysis is done on monthly basis. However, they include several variables that are based on daily/weekly data. That's why we need to start with daily data. 

Input of this notebook: CRSP daily data. 'daily.csv'


Output of this notebook: On monthly basis, features like:
- Monthly (lagged) variance (idiosyncratic + market) - calculated from daily returns
- bull/bear runs (duration) in days (before month end) and weeks (before month end)


# Daily data

## Initial Data Cleaning

### Load data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import the parquet library
import pyarrow.parquet as pq

In [2]:
# Load the data
df = pd.read_csv("daily.csv", parse_dates = True)


c:\Users\tomta\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head(20)

,PERMNO,date,SHRCD,DLRET,RET
0,10000,1986-01-06,NaN,NaN,NaN
1,10000,1986-01-07,10.0,NaN,C
2,10000,1986-01-08,10.0,NaN,-0.024390
3,10000,1986-01-09,10.0,NaN,0.000000
4,10000,1986-01-10,10.0,NaN,0.000000
5,10000,1986-01-13,10.0,NaN,0.050000
6,10000,1986-01-14,10.0,NaN,0.047619
7,10000,1986-01-15,10.0,NaN,0.045455
8,10000,1986-01-16,10.0,NaN,0.043478
9,10000,1986-01-17,10.0,NaN,0.000000


In [4]:
df.shape

(101919488, 5)

### Common and ordinary stock only

In [5]:
# drop rows that do not have SHRCD = 10 or 11

df = df[df['SHRCD'].isin([10, 11])]

In [6]:
df.shape

(79560924, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79560924 entries, 1 to 101919487
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   PERMNO  int64  
 1   date    object 
 2   SHRCD   float64
 3   DLRET   object 
 4   RET     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.6+ GB


In [8]:
# drop the SHRCD column
df = df.drop(columns = ['SHRCD'])

df.head()

,PERMNO,date,DLRET,RET
1,10000,1986-01-07,NaN,C
2,10000,1986-01-08,NaN,-0.024390
3,10000,1986-01-09,NaN,0.000000
4,10000,1986-01-10,NaN,0.000000
5,10000,1986-01-13,NaN,0.050000


### Initial cleaning of returns, adjusting for delisting returns

In [9]:
# count rows where RET is not a number
print(df['RET'].isna().sum())

# count rows where RET == C
print(df['RET'].value_counts()['C'])

442988
29748


In [10]:
df = df[df['RET'] != 'C']
df.shape

(79531176, 4)

In [11]:
# convert RET to numeric
df['RET'] = pd.to_numeric(df['RET'], errors='coerce')

In [12]:
# Convert 'RET' and 'DLRET' to numeric values, coercing errors into NaN
df['DLRET'] = pd.to_numeric(df['DLRET'], errors='coerce')

In [13]:
df.describe()

,PERMNO,DLRET,RET
count,7.953118e+07,19056.000000,7.784353e+07
mean,4.963722e+04,-0.035032,8.199555e-04
std,2.837886e+04,0.319376,4.478152e-02
min,1.000000e+04,-1.000000,-9.716980e-01
25%,2.085300e+04,-0.011765,-1.232400e-02
50%,4.829100e+04,0.000614,0.000000e+00
75%,7.831900e+04,0.017614,1.151500e-02
max,9.343600e+04,18.636364,1.900000e+01


In [14]:
# count missing values in each column
df.isna().sum()

PERMNO           0
date             0
DLRET     79512120
RET        1687647
dtype: int64

In [15]:
# replace -1 in 'DLRET' with nan
df['DLRET'] = df['DLRET'].replace(-1, np.nan)


In [17]:
# replace RET with DLRET where DLRET is not NA
df['RET'] = np.where(df['DLRET'].notna(), df['DLRET'], df['RET'])

# drop rows where RET is NA
df = df.dropna(subset=['RET'])

In [18]:
# count missing values in each column
df.isna().sum()

PERMNO           0
date             0
DLRET     77824841
RET              0
dtype: int64

### Introduce log returns

In [19]:
# Calculate log returns for 'RET''. Note: log(1+r) for each return
df['logRet'] = np.log(1 + df['RET'])

# If you want to convert back to a standard return format from log return:
# df['Total_Return_Converted'] = np.exp(df['Total_Log_Return']) - 1

In [20]:
df.head()

,PERMNO,date,DLRET,RET,logRet
2,10000,1986-01-08,NaN,-0.024390,-0.024692
3,10000,1986-01-09,NaN,0.000000,0.000000
4,10000,1986-01-10,NaN,0.000000,0.000000
5,10000,1986-01-13,NaN,0.050000,0.048790
6,10000,1986-01-14,NaN,0.047619,0.046520


In [22]:
# drop the columns that are not needed (DLRET, RET, Log_RET, Log_DLRET)

df = df.drop(['DLRET', 'RET'], axis = 1)


In [23]:
df.head()

,PERMNO,date,logRet
2,10000,1986-01-08,-0.024692
3,10000,1986-01-09,0.000000
4,10000,1986-01-10,0.000000
5,10000,1986-01-13,0.048790
6,10000,1986-01-14,0.046520


In [25]:
df.dtypes  

PERMNO      int64
date       object
logRet    float64
dtype: object

In [26]:
# set the date column to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [27]:
df.dtypes

PERMNO             int64
date      datetime64[ns]
logRet           float64
dtype: object

In [29]:
# save the data to parquet - in case the code breaks and you need to start from here
# df.to_parquet("daily_common_stock_logret.parquet")

## Checkpoint 1 - so far we have cleaned logreturns adjusted for delistings
In this part, calculate bull/bear runs in days

### In this part, calculate bull/bear runs in days

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import the parquet library
import pyarrow.parquet as pq

In [2]:
# load the data from parquet
df = pd.read_parquet("daily_common_stock_logret.parquet")

In [3]:
df.head()

,PERMNO,date,logRet
2,10000,1986-01-08,-0.024692
3,10000,1986-01-09,0.000000
4,10000,1986-01-10,0.000000
5,10000,1986-01-13,0.048790
6,10000,1986-01-14,0.046520


Encoding the bull and bear runs in days

In [4]:
# create a new column inc for the increase in price that takes the value 1 if the log return is positive, 0 otherwise
df['inc'] = (df['logRet'] > 0).astype(int)

# create a new column dec for the decrease in price that takes the value 1 if the log return is negative, 0 otherwise
df['dec'] = (df['logRet'] < 0).astype(int)

df.head(5)

,PERMNO,date,logRet,inc,dec
2,10000,1986-01-08,-0.024692,0,1
3,10000,1986-01-09,0.000000,0,0
4,10000,1986-01-10,0.000000,0,0
5,10000,1986-01-13,0.048790,1,0
6,10000,1986-01-14,0.046520,1,0
7,10000,1986-01-15,0.044452,1,0
8,10000,1986-01-16,0.042559,1,0
9,10000,1986-01-17,0.000000,0,0
10,10000,1986-01-20,0.000000,0,0
11,10000,1986-01-21,0.000000,0,0


In [5]:
# Sort the DataFrame by 'PERMNO' and 'date'
df = df.sort_values(by=['PERMNO', 'date'])

# Identify when a trend changes or when it's a new stock
df['trend_change'] = (
    (df['inc'] != df['inc'].shift(1)) | 
    (df['dec'] != df['dec'].shift(1)) | 
    (df['PERMNO'] != df['PERMNO'].shift(1))
).astype(int)

df.head(5)

,PERMNO,date,logRet,inc,dec,trend_change
2,10000,1986-01-08,-0.024692,0,1,1
3,10000,1986-01-09,0.000000,0,0,1
4,10000,1986-01-10,0.000000,0,0,0
5,10000,1986-01-13,0.048790,1,0,1
6,10000,1986-01-14,0.046520,1,0,0
7,10000,1986-01-15,0.044452,1,0,0
8,10000,1986-01-16,0.042559,1,0,0
9,10000,1986-01-17,0.000000,0,0,1
10,10000,1986-01-20,0.000000,0,0,0
11,10000,1986-01-21,0.000000,0,0,0


In [6]:
# Create a segment identifier for consecutive trends - later we will use this to group the data
df['segment_id'] = df.groupby('PERMNO')['trend_change'].cumsum()

df.head(5)

,PERMNO,date,logRet,inc,dec,trend_change,segment_id
2,10000,1986-01-08,-0.024692,0,1,1,1
3,10000,1986-01-09,0.000000,0,0,1,2
4,10000,1986-01-10,0.000000,0,0,0,2
5,10000,1986-01-13,0.048790,1,0,1,3
6,10000,1986-01-14,0.046520,1,0,0,3


In [7]:
# Calculate lengths of bull and bear segments 
df['bull_days'] = df.groupby(['PERMNO', 'segment_id'])['inc'].cumsum()

# Reset 'bull_days' and 'bear_days' to 0 on days there's no increase or decrease
df.loc[df['inc'] == 0, 'bull_days'] = 0


df.head(10)


,PERMNO,date,logRet,inc,dec,trend_change,segment_id,bull_days
2,10000,1986-01-08,-0.024692,0,1,1,1,0
3,10000,1986-01-09,0.000000,0,0,1,2,0
4,10000,1986-01-10,0.000000,0,0,0,2,0
5,10000,1986-01-13,0.048790,1,0,1,3,1
6,10000,1986-01-14,0.046520,1,0,0,3,2
7,10000,1986-01-15,0.044452,1,0,0,3,3
8,10000,1986-01-16,0.042559,1,0,0,3,4
9,10000,1986-01-17,0.000000,0,0,1,4,0
10,10000,1986-01-20,0.000000,0,0,0,4,0
11,10000,1986-01-21,0.000000,0,0,0,4,0


In [8]:
# same for bear days
df['bear_days'] = df.groupby(['PERMNO', 'segment_id'])['dec'].cumsum()
df.loc[df['dec'] == 0, 'bear_days'] = 0

df.head(20)

,PERMNO,date,logRet,inc,dec,trend_change,segment_id,bull_days,bear_days
2,10000,1986-01-08,-0.024692,0,1,1,1,0,1
3,10000,1986-01-09,0.000000,0,0,1,2,0,0
4,10000,1986-01-10,0.000000,0,0,0,2,0,0
5,10000,1986-01-13,0.048790,1,0,1,3,1,0
6,10000,1986-01-14,0.046520,1,0,0,3,2,0
7,10000,1986-01-15,0.044452,1,0,0,3,3,0
8,10000,1986-01-16,0.042559,1,0,0,3,4,0
9,10000,1986-01-17,0.000000,0,0,1,4,0,0
10,10000,1986-01-20,0.000000,0,0,0,4,0,0
11,10000,1986-01-21,0.000000,0,0,0,4,0,0


In [9]:
# subtract 1 from bull_days and from bear_days - we look in the past by one month
# this is to avoid look-ahead bias - we dont include the current day/week/month

df['bull_days'] = df['bull_days'] - 1
df['bear_days'] = df['bear_days'] - 1

# if bull_days is negative, set it to 0
df.loc[df['bull_days'] < 0, 'bull_days'] = 0
df.loc[df['bear_days'] < 0, 'bear_days'] = 0

# follow the paper:
# if bear_days is > 12, set it to 12, same for bull_days
df.loc[df['bull_days'] > 12, 'bull_days'] = 12
df.loc[df['bear_days'] > 12, 'bear_days'] = 12

df.head(20)

,PERMNO,date,logRet,inc,dec,trend_change,segment_id,bull_days,bear_days
2,10000,1986-01-08,-0.024692,0,1,1,1,0,0
3,10000,1986-01-09,0.000000,0,0,1,2,0,0
4,10000,1986-01-10,0.000000,0,0,0,2,0,0
5,10000,1986-01-13,0.048790,1,0,1,3,0,0
6,10000,1986-01-14,0.046520,1,0,0,3,1,0
7,10000,1986-01-15,0.044452,1,0,0,3,2,0
8,10000,1986-01-16,0.042559,1,0,0,3,3,0
9,10000,1986-01-17,0.000000,0,0,1,4,0,0
10,10000,1986-01-20,0.000000,0,0,0,4,0,0
11,10000,1986-01-21,0.000000,0,0,0,4,0,0


In [10]:
# keep only the columns we need - bull and bear days
df.drop(['inc', 'dec', 'trend_change', 'segment_id'], axis = 1, inplace = True)

df.head(20)
# by visual inspection, the data looks good

,PERMNO,date,logRet,bull_days,bear_days
2,10000,1986-01-08,-0.024692,0,0
3,10000,1986-01-09,0.000000,0,0
4,10000,1986-01-10,0.000000,0,0
5,10000,1986-01-13,0.048790,0,0
6,10000,1986-01-14,0.046520,1,0
7,10000,1986-01-15,0.044452,2,0
8,10000,1986-01-16,0.042559,3,0
9,10000,1986-01-17,0.000000,0,0
10,10000,1986-01-20,0.000000,0,0
11,10000,1986-01-21,0.000000,0,0


In [11]:
# save the data to parquet - in case the code breaks and you need to start from here
df.to_parquet("daily_v1.parquet")

## Checkpoint 2 - we have bull and bear runs in days


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import the parquet library
import pyarrow.parquet as pq

In [2]:
df = pd.read_parquet("daily_v1.parquet")

#### Additional cleaning... forgot to do this before - should have done that earlier - but we include only stocks with at least one year of available data

In [3]:
# Count unique months for each PERMNO as a proxy for filtering stocks with at least a year of data
unique_months_per_stock = df.groupby('PERMNO').apply(lambda x: x['date'].dt.to_period('M').nunique())


# Filter PERMNOs with at least 12 unique months
stocks_with_year_data = unique_months_per_stock[unique_months_per_stock >= 12].index

# Filter the original DataFrame
df = df[df['PERMNO'].isin(stocks_with_year_data)]

In [4]:
df.to_parquet("daily_v1_1.parquet")

In [ ]:
df = pd.read_parquet("daily_v1_1.parquet")

### Keep bull/bear varibales only for month ends and save it for later use on monthly data

In [5]:
# unnecessary to keep all the days - we need only month end data - save it to be merged later
bull_bear = df.groupby(['PERMNO', pd.Grouper(key='date', freq='M')]).agg('last').reset_index()

# keep columns needed for merging wiht the monthly data
bull_bear = bull_bear[['PERMNO', 'date', 'bull_days', 'bear_days']]

print(bull_bear.head())

# save the data to parquet
bull_bear.to_parquet("bull_bear.parquet")

   PERMNO       date  bull_days  bear_days
0   10000 1986-01-31          0          0
1   10000 1986-02-28          0          0
2   10000 1986-03-31          0          0
3   10000 1986-04-30          0          1
4   10000 1986-05-31          0          0


,PERMNO,date,logRet
2,10000,1986-01-08,-0.024692
3,10000,1986-01-09,0.000000
4,10000,1986-01-10,0.000000
5,10000,1986-01-13,0.048790
6,10000,1986-01-14,0.046520


In [ ]:
# drop the columns that are not needed (bull and bear) from df - they are saved in the parquet file and ready to be merged with the monthly data
df = df.drop(['bull_days', 'bear_days'], axis = 1)
df.head()   

### Get market returns from FF

In [3]:
# load 'FF3_day.csv', careful the header is in the fifth row, data starts from the sixth row, date is in the first column

ff3 = pd.read_csv("FF3_day.csv", header = 3, parse_dates = True)

ff3.tail()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
25645,20231226,0.48,0.69,0.46,0.021
25646,20231227,0.16,0.14,0.12,0.021
25647,20231228,-0.01,-0.36,0.03,0.021
25648,20231229,-0.43,-1.12,-0.37,0.021
25649,Copyright 2023 Kenneth R. French,NaN,NaN,NaN,NaN


In [4]:
# drop the last row

ff3 = ff3[:-1]
ff3.tail()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
25644,20231222,0.21,0.64,0.09,0.021
25645,20231226,0.48,0.69,0.46,0.021
25646,20231227,0.16,0.14,0.12,0.021
25647,20231228,-0.01,-0.36,0.03,0.021
25648,20231229,-0.43,-1.12,-0.37,0.021


In [5]:
# the first column is the date, rename this column to 'date'
ff3 = ff3.rename(columns = {'Unnamed: 0': 'date'})

# it contains date in the format year, month, day, convert it to a date format

ff3['date'] = pd.to_datetime(ff3['date'], format='%Y%m%d')

ff3.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-01,0.10,-0.25,-0.27,0.009
1,1926-07-02,0.45,-0.33,-0.06,0.009
2,1926-07-06,0.17,0.30,-0.39,0.009
3,1926-07-07,0.09,-0.58,0.02,0.009
4,1926-07-08,0.21,-0.38,0.19,0.009


In [6]:
# check for data types

ff3.dtypes

date      datetime64[ns]
Mkt-RF           float64
SMB              float64
HML              float64
RF               float64
dtype: object

In [7]:
# create 'Mkt' column as a sum of Mkt-RF and RF             
ff3['Mkt'] = ff3['Mkt-RF'] + ff3['RF']
ff3.head()

,date,Mkt-RF,SMB,HML,RF,Mkt
0,1926-07-01,0.10,-0.25,-0.27,0.009,0.109
1,1926-07-02,0.45,-0.33,-0.06,0.009,0.459
2,1926-07-06,0.17,0.30,-0.39,0.009,0.179
3,1926-07-07,0.09,-0.58,0.02,0.009,0.099
4,1926-07-08,0.21,-0.38,0.19,0.009,0.219


In [8]:
market = ff3[['date', 'Mkt', 'Mkt-RF']]

# divide by 100
market['Mkt'] = market['Mkt'] / 100
market['Mkt-RF'] = market['Mkt-RF'] / 100

# note: FF data are in percent - should be divided by 100. However, technically, it is not necessary since it is only used for CAPM prediction
# CAPM beta will be 100 times smaller than they should, but the prediction will be the same

market.head()

<ipython-input-8-cd72ddb32df8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market['Mkt'] = market['Mkt'] / 100
<ipython-input-8-cd72ddb32df8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market['Mkt-RF'] = market['Mkt-RF'] / 100


,date,Mkt,Mkt-RF
0,1926-07-01,0.00109,0.0010
1,1926-07-02,0.00459,0.0045
2,1926-07-06,0.00179,0.0017
3,1926-07-07,0.00099,0.0009
4,1926-07-08,0.00219,0.0021


In [9]:
# save the data to parquet
market.to_parquet("market_daily.parquet")

In [13]:
df.head()

,PERMNO,date,logRet
2,10000,1986-01-08,-0.024692
3,10000,1986-01-09,0.000000
4,10000,1986-01-10,0.000000
5,10000,1986-01-13,0.048790
6,10000,1986-01-14,0.046520


In [14]:
# create a new column in which there is a simple return (take exp of log return and subtract 1)

df['ret'] = np.exp(df['logRet']) - 1

df.head()

,PERMNO,date,logRet,ret
2,10000,1986-01-08,-0.024692,-0.024390
3,10000,1986-01-09,0.000000,0.000000
4,10000,1986-01-10,0.000000,0.000000
5,10000,1986-01-13,0.048790,0.050000
6,10000,1986-01-14,0.046520,0.047619


In [15]:
# Merge the market log return with the original DataFrame

df = df.merge(market, on='date')

df.head()


,PERMNO,date,logRet,ret,Mkt,Mkt-RF
0,10000,1986-01-08,-0.024692,-0.024390,-2.135,-2.16
1,10015,1986-01-08,0.000000,0.000000,-2.135,-2.16
2,10031,1986-01-08,0.000000,0.000000,-2.135,-2.16
3,10057,1986-01-08,-0.035092,-0.034483,-2.135,-2.16
4,10066,1986-01-08,0.000000,0.000000,-2.135,-2.16


In [16]:
df.isna().sum()

PERMNO    0
date      0
logRet    0
ret       0
Mkt       0
Mkt-RF    0
dtype: int64

In [17]:
df.shape

(77608075, 6)

In [18]:
# save the data to parquet
df.to_parquet("daily_v2.parquet")

## Checkpoin 3: ready to Calculate idiosyncratic volatility: variance of difference between CAPM prediction and actual return

### Make the CAPM predictions and calculate differences

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import the parquet library
import pyarrow.parquet as pq

import statsmodels.api as sm

from tqdm import tqdm
from tqdm.auto import tqdm

In [2]:
df = pd.read_parquet("daily_v2.parquet")

In [3]:
# we estimate CAPM for each year and stock based on daily data
# create a new column for year - we will use this to group the data
df['year'] = df['date'].dt.to_period('Y')
df.head()

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,year
0,10000,1986-01-08,-0.024692,-0.024390,-2.135,-2.16,1986
1,10015,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986
2,10031,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986
3,10057,1986-01-08,-0.035092,-0.034483,-2.135,-2.16,1986
4,10066,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986


In [4]:
# Add a constant to the whole dataset at once
df['const'] = 1
df.head()

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,year,const
0,10000,1986-01-08,-0.024692,-0.024390,-2.135,-2.16,1986,1
1,10015,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986,1
2,10031,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986,1
3,10057,1986-01-08,-0.035092,-0.034483,-2.135,-2.16,1986,1
4,10066,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986,1


### Run on cloud, don't ruin your PC

In [9]:
# split the df into df1, df2, df3 based on the PERMNO column (df1 contains permno < 30000, df2 contains permno >= 30000 but less than 60000, df3 contains permno >= 60000)
df1 = df[df['PERMNO'] < 30000]
df2 = df[(df['PERMNO'] >= 30000) & (df['PERMNO'] < 70000)]
df3 = df[df['PERMNO'] >= 70000]





In [6]:
print(df1.shape)
print(df2.shape)
print(df3.shape)

(27603108, 8)
(17886533, 8)
(32118434, 8)


#### do it by parts - df1

In [2]:
# load df1
# df1 = pd.read_parquet("df1.parquet")


#### Estimate yearly CAPM models for each stock and make prdictions

In [12]:
# Function to apply OLS regression within each group and predict returns
def estimate_and_predict(group):
    # Ensure the group has enough data points
    if len(group) > 1:
        # Use the 'const' and 'Mkt-RF' columns as independent variables - CAPM model
        X = group[['const', 'Mkt-RF']]
        y = group['ret']  # Dependent variable
        model = sm.OLS(y, X).fit()  # Fit model
        group['CAPM_ret'] = model.predict(X)  # Predict returns with the model
    else:
        group['CAPM_ret'] = pd.NA
    return group


In [13]:
tqdm.pandas()

# Apply the function with progress tracking
# Note: progress_apply() comes from tqdm integration with pandas
df1 = df1.groupby(['PERMNO', 'year']).progress_apply(estimate_and_predict)

100%|██████████| 114440/114440 [09:50<00:00, 193.96it/s]


In [15]:
df1.head()

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,year,const,CAPM_ret
0,10000,1986-01-08,-0.024692,-0.024390,-2.135,-2.16,1986,1,-0.019082
1,10015,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986,1,-0.017987
2,10031,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986,1,-0.001664
3,10057,1986-01-08,-0.035092,-0.034483,-2.135,-2.16,1986,1,-0.022101
4,10066,1986-01-08,0.000000,0.000000,-2.135,-2.16,1986,1,-0.000844


In [19]:
# drop year and const columns
df1 = df1.drop(['year', 'const'], axis = 1)

# calculate the excess return (difference between the actual return and the predicted return)
df1['exc_ret'] = df1['ret'] - df1['CAPM_ret']

In [20]:
df1.isna().sum()

PERMNO       0
date         0
logRet       0
ret          0
Mkt          0
Mkt-RF       0
CAPM_ret    49
exc_ret     49
dtype: int64

In [21]:
df1.head()

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,CAPM_ret,exc_ret
0,10000,1986-01-08,-0.024692,-0.024390,-2.135,-2.16,-0.019082,-0.005308
1,10015,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.017987,0.017987
2,10031,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.001664,0.001664
3,10057,1986-01-08,-0.035092,-0.034483,-2.135,-2.16,-0.022101,-0.012382
4,10066,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.000844,0.000844


In [22]:
# save the data to parquet
df1.to_parquet("CAPM_1.parquet")

#### df2

In [23]:
tqdm.pandas()

# Apply the function with progress tracking
# Note: progress_apply() comes from tqdm integration with pandas
df2 = df2.groupby(['PERMNO', 'year']).progress_apply(estimate_and_predict)

100%|██████████| 99020/99020 [08:17<00:00, 199.09it/s]


In [24]:
# drop year and const columns
# df2 = df2.drop(['year', 'const'], axis = 1)

# calculate the excess return
df2['exc_ret'] = df2['ret'] - df2['CAPM_ret']

df2.head()

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,CAPM_ret,exc_ret
1400,30024,1986-01-08,0.051294,0.052632,-2.135,-2.16,-0.011028,0.063660
1401,30031,1986-01-08,-0.007380,-0.007353,-2.135,-2.16,-0.012376,0.005023
1402,30059,1986-01-08,0.021506,0.021739,-2.135,-2.16,-0.017760,0.039499
1403,30067,1986-01-08,-0.090972,-0.086957,-2.135,-2.16,-0.034901,-0.052056
1404,30075,1986-01-08,-0.005970,-0.005952,-2.135,-2.16,-0.018696,0.012744


In [25]:
df2.isna().sum()

PERMNO       0
date         0
logRet       0
ret          0
Mkt          0
Mkt-RF       0
CAPM_ret    52
exc_ret     52
dtype: int64

In [26]:
df2.to_parquet("CAPM_2.parquet")

#### df3

In [27]:
tqdm.pandas()

# Apply the function with progress tracking
# Note: progress_apply() comes from tqdm integration with pandas
df3 = df3.groupby(['PERMNO', 'year']).progress_apply(estimate_and_predict)

100%|██████████| 118914/118914 [10:39<00:00, 186.07it/s]


In [28]:
# drop year and const columns
df3 = df3.drop(['year', 'const'], axis = 1)

# calculate the excess return
df3['exc_ret'] = df3['ret'] - df3['CAPM_ret']

df3.head()

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,CAPM_ret,exc_ret
4517,70018,1986-01-08,-0.028171,-0.027778,-2.135,-2.16,-0.022280,-0.005498
4518,70026,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.003096,0.003096
4519,70077,1986-01-08,-0.031416,-0.030928,-2.135,-2.16,-0.031814,0.000886
4520,70093,1986-01-08,0.039221,0.040000,-2.135,-2.16,-0.011415,0.051415
4521,70106,1986-01-08,-0.006270,-0.006250,-2.135,-2.16,-0.015951,0.009701


In [29]:
df3.isna().sum()

PERMNO       0
date         0
logRet       0
ret          0
Mkt          0
Mkt-RF       0
CAPM_ret    73
exc_ret     73
dtype: int64

In [30]:
df3.to_parquet("CAPM_3.parquet")    

#### bind them back together

In [31]:
# bind df1 and df2 back together
df = pd.concat([df1, df2, df3])
df.shape

(77608075, 8)

In [32]:
df.head(10)

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,CAPM_ret,exc_ret
0,10000,1986-01-08,-0.024692,-0.024390,-2.135,-2.16,-0.019082,-0.005308
1,10015,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.017987,0.017987
2,10031,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.001664,0.001664
3,10057,1986-01-08,-0.035092,-0.034483,-2.135,-2.16,-0.022101,-0.012382
4,10066,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.000844,0.000844
5,10074,1986-01-08,0.000000,0.000000,-2.135,-2.16,0.005415,-0.005415
6,10082,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.014951,0.014951
7,10103,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.001531,0.001531
8,10137,1986-01-08,-0.022223,-0.021978,-2.135,-2.16,-0.010715,-0.011263
9,10145,1986-01-08,-0.013352,-0.013263,-2.135,-2.16,-0.018212,0.004949


In [33]:
df.to_parquet("daily_v2_1.parquet")

### Have the CAPM residuals. Next, calculate monthly variances

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import the parquet library
import pyarrow.parquet as pq

In [2]:
df = pd.read_parquet("daily_v2_1.parquet")

In [34]:
df.isna().sum()

PERMNO        0
date          0
logRet        0
ret           0
Mkt           0
Mkt-RF        0
CAPM_ret    174
exc_ret     174
dtype: int64

In [35]:
df.tail()

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,CAPM_ret,exc_ret
77608070,86530,1983-09-19,0.0,0.0,1.036,1.0,0.005055,-0.005055
77608071,89790,1983-09-19,0.0,0.0,1.036,1.0,0.005071,-0.005071
77608072,90705,1983-09-19,0.0,0.0,1.036,1.0,0.018598,-0.018598
77608073,90975,1983-09-19,0.0,0.0,1.036,1.0,0.006307,-0.006307
77608074,92946,1983-09-19,0.0,0.0,1.036,1.0,0.010176,-0.010176


#### Monthly market variance - this is on top of the Base model. Additional Market variable

In [10]:
market.head()

,date,Mkt,Mkt-RF
0,1926-07-01,0.00109,0.0010
1,1926-07-02,0.00459,0.0045
2,1926-07-06,0.00179,0.0017
3,1926-07-07,0.00099,0.0009
4,1926-07-08,0.00219,0.0021


In [11]:
# Group by year and month, then calculate variance of 'mkt' for each group
market_varianceM = market.groupby(pd.Grouper(key='date', freq='M'))['Mkt'].var().reset_index()

# Rename the 'mkt_logRet' column to 'monthly_variance' to reflect the content
market_varianceM.rename(columns={'Mkt': 'mVarM'}, inplace=True)

market_varianceM.head()

,date,mVarM
0,1926-07-31,0.000021
1,1926-08-31,0.000035
2,1926-09-30,0.000025
3,1926-10-31,0.000070
4,1926-11-30,0.000015


In [12]:
# save the data to parquet
market_varianceM.to_parquet("market_variance_month.parquet")

### Monthly Idiosyncratic variance

In [37]:
# Group by 'PERMNO' and month, then calculate variance of 'excessRet' for each group
monthly_variance_stockM = df.groupby(['PERMNO', pd.Grouper(key='date', freq='M')])['exc_ret'].var().reset_index()

# Rename the 'excessRet' column to 'monthly_variance' to reflect the content
monthly_variance_stockM.rename(columns={'exc_ret': 'sVarM'}, inplace=True)


In [38]:
monthly_variance_stockM.head()

,PERMNO,date,sVarM
0,10000,1986-01-31,0.004058
1,10000,1986-02-28,0.000973
2,10000,1986-03-31,0.001993
3,10000,1986-04-30,0.000163
4,10000,1986-05-31,0.001569


In [39]:
# save the data to parquet
monthly_variance_stockM.to_parquet("monthly_variance_stockM.parquet")


## Checkpoint 4: We have all daily variables. Let's continue with weekly

###  Prepare for the daily => weekly aggregation

In [49]:
df.head()   

,PERMNO,date,logRet,ret,Mkt,Mkt-RF,CAPM_ret,exc_ret
0,10000,1986-01-08,-0.024692,-0.024390,-2.135,-2.16,-0.019082,-0.005308
1,10015,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.017987,0.017987
2,10031,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.001664,0.001664
3,10057,1986-01-08,-0.035092,-0.034483,-2.135,-2.16,-0.022101,-0.012382
4,10066,1986-01-08,0.000000,0.000000,-2.135,-2.16,-0.000844,0.000844


In [50]:
# keep only the columns we need - we cumsum the log returns to get weekly returns
df = df.drop(columns = ['Mkt', 'Mkt-RF', 'ret', 'CAPM_ret', 'exc_ret'])

df.head()

,PERMNO,date,logRet
0,10000,1986-01-08,-0.024692
1,10015,1986-01-08,0.000000
2,10031,1986-01-08,0.000000
3,10057,1986-01-08,-0.035092
4,10066,1986-01-08,0.000000


In [51]:
df.to_parquet("daily_v4.parquet")

### All daily variables are ready => Transform daily to weekly and create week-based variables

In [38]:
df = pd.read_parquet("daily_v4.parquet")

In [61]:
df.sort_values(by = ['PERMNO', 'date'], inplace = True)

In [64]:
df.head(20)

,PERMNO,logRet
date,,
1986-01-08,10000,-0.024692
1986-01-09,10000,0.000000
1986-01-10,10000,0.000000
1986-01-13,10000,0.048790
1986-01-14,10000,0.046520
1986-01-15,10000,0.044452
1986-01-16,10000,0.042559
1986-01-17,10000,0.000000
1986-01-20,10000,0.000000


In [62]:
df.set_index('date', inplace=True)

# Define a function to transform daily log returns into weekly log returns
def weekly_log_returns(group):
    # Resample to weekly, assuming the week starts on Monday and using the sum of log returns
    week = group['logRet'].resample('W').sum()
    return week

# Group by 'PERMNO' and apply the transformation function
week = df.groupby('PERMNO').apply(weekly_log_returns).reset_index()

# Rename columns appropriately
week.rename(columns={'logRet': 'W_logRet'}, inplace=True)

week.head()

,PERMNO,date,W_logRet
0,10000,1986-01-12,-2.469236e-02
1,10000,1986-01-19,1.823217e-01
2,10000,1986-01-26,3.334919e-01
3,10000,1986-02-02,4.380215e-02
4,10000,1986-02-09,3.496800e-08


In [65]:
# save the data to parquet
week.to_parquet("weekly_v1.parquet")

### Bull and Bear market duration in weeks

Same code as with daily data...

In [66]:
# Create a new column 'inc' for the increase in price that takes the value 1 if the log return is positive, 0 otherwise
week['inc'] = (week['W_logRet'] > 0).astype(int)

# Create a new column 'dec' for the decrease in price that takes the value 1 if the log return is negative, 0 otherwise
week['dec'] = (week['W_logRet'] < 0).astype(int)

# Sort the DataFrame by 'PERMNO' and 'date'
week = week.sort_values(by=['PERMNO', 'date'])

# Identify when a trend changes or when it's a new stock
week['trend_change'] = (
    (week['inc'] != week['inc'].shift(1)) | 
    (week['dec'] != week['dec'].shift(1)) | 
    (week['PERMNO'] != week['PERMNO'].shift(1))
).astype(int)

# Create a segment identifier for consecutive trends
week['segment_id'] = week.groupby('PERMNO')['trend_change'].cumsum()

# Calculate lengths of bull and bear segments
week['bull_W'] = week.groupby(['PERMNO', 'segment_id'])['inc'].cumsum()

# Reset 'bull_W' and 'bear_W' to 0 on days there's no increase or decrease
week.loc[week['inc'] == 0, 'bull_W'] = 0

week['bear_W'] = week.groupby(['PERMNO', 'segment_id'])['dec'].cumsum()
week.loc[week['dec'] == 0, 'bear_W'] = 0

# Subtract 1 from 'bull_W' and from 'bear_W'
week['bull_W'] = week['bull_W'] - 1
week['bear_W'] = week['bear_W'] - 1

# If 'bull_W' is negative, set it to 0
week.loc[week['bull_W'] < 0, 'bull_W'] = 0
week.loc[week['bear_W'] < 0, 'bear_W'] = 0

# If 'bear_W' is > 12, set it to 12, same for 'bull_W'
week.loc[week['bull_W'] > 12, 'bull_W'] = 12
week.loc[week['bear_W'] > 12, 'bear_W'] = 12

week.head(20)

,PERMNO,date,W_logRet,inc,dec,trend_change,segment_id,bull_W,bear_W
0,10000,1986-01-12,-2.469236e-02,0,1,1,1,0,0
1,10000,1986-01-19,1.823217e-01,1,0,1,2,0,0
2,10000,1986-01-26,3.334919e-01,1,0,0,2,1,0
3,10000,1986-02-02,4.380215e-02,1,0,0,2,2,0
4,10000,1986-02-09,3.496800e-08,1,0,0,2,3,0
5,10000,1986-02-16,-2.166147e-02,0,1,1,3,0,0
6,10000,1986-02-23,-1.492958e-01,0,1,0,3,0,1
7,10000,1986-03-02,-1.262929e-01,0,1,0,3,0,2
8,10000,1986-03-09,3.114356e-01,1,0,1,4,0,0
9,10000,1986-03-16,7.551007e-07,1,0,0,4,1,0


In [67]:
# drop = ['inc', 'dec', 'trend_change', 'segment_id']
week.drop(['inc', 'dec', 'trend_change', 'segment_id'], axis = 1, inplace = True)

week.head(5)

,PERMNO,date,W_logRet,bull_W,bear_W
0,10000,1986-01-12,-2.469236e-02,0,0
1,10000,1986-01-19,1.823217e-01,0,0
2,10000,1986-01-26,3.334919e-01,1,0
3,10000,1986-02-02,4.380215e-02,2,0
4,10000,1986-02-09,3.496800e-08,3,0


In [68]:
# again, we need only month end data - save it to be merged later
bull_bear = week.groupby(['PERMNO', pd.Grouper(key='date', freq='M')]).agg('last').reset_index()
bull_bear = bull_bear[['PERMNO', 'date', 'bull_W', 'bear_W']]
bull_bear.head()

,PERMNO,date,bull_W,bear_W
0,10000,1986-01-31,1,0
1,10000,1986-02-28,0,1
2,10000,1986-03-31,0,0
3,10000,1986-04-30,0,0
4,10000,1986-05-31,0,0


In [69]:
# save the data to parquet
bull_bear.to_parquet("bull_bear_weekly.parquet")

## Import and handle weekly variances


In [70]:
# drop the columns that are not needed (bull and bear) from week
week = week.drop(['bull_W', 'bear_W'], axis = 1)
week.head()

,PERMNO,date,W_logRet
0,10000,1986-01-12,-2.469236e-02
1,10000,1986-01-19,1.823217e-01
2,10000,1986-01-26,3.334919e-01
3,10000,1986-02-02,4.380215e-02
4,10000,1986-02-09,3.496800e-08


In [71]:
# save the data to parquet as weekly_v2_beforeLagged.parquet

week2 = pd.read_parquet("weekly_v1.parquet")
week2.head()

,PERMNO,date,W_logRet
0,10000,1986-01-12,-2.469236e-02
1,10000,1986-01-19,1.823217e-01
2,10000,1986-01-26,3.334919e-01
3,10000,1986-02-02,4.380215e-02
4,10000,1986-02-09,3.496800e-08


# Monthly data - Merge all variables created here to one dataset with monthly frequency

### Load all data files that contain variables on monthly basis

In [13]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import the parquet library
import pyarrow.parquet as pq

In [14]:
month_features = pd.read_parquet("bull_bear.parquet")

In [15]:
# resample the data to monthly frequency keeping the last observation 

month_features.head()

,PERMNO,date,bull_days,bear_days
0,10000,1986-01-31,0,0
1,10000,1986-02-28,0,0
2,10000,1986-03-31,0,0
3,10000,1986-04-30,0,1
4,10000,1986-05-31,0,0


In [16]:
month_features.rename(columns={'bull_days': 'bull_D', 'bear_days': 'bear_D'}, inplace=True)

In [17]:
# load "monthly_variance_stockM.parquet" and "market_variance_month.parquet" and bull_bear.parquet and bull_bear_weekly.parquet

monthly_variance_stockM = pd.read_parquet("monthly_variance_stockM.parquet")
market_varianceM = pd.read_parquet("market_variance_month.parquet")
DurW = pd.read_parquet("bull_bear_weekly.parquet")

In [18]:
monthly_variance_stockM.head()

,PERMNO,date,sVarM
0,10000,1986-01-31,0.004058
1,10000,1986-02-28,0.000973
2,10000,1986-03-31,0.001993
3,10000,1986-04-30,0.000163
4,10000,1986-05-31,0.001569


In [19]:
DurW.head()

,PERMNO,date,bull_W,bear_W
0,10000,1986-01-31,1,0
1,10000,1986-02-28,0,1
2,10000,1986-03-31,0,0
3,10000,1986-04-30,0,0
4,10000,1986-05-31,0,0


In [20]:
# merge DurW and DurD on PERMNO and date
month_features = month_features.merge(DurW, on = ['PERMNO', 'date'])

# merge temp and monthly_variance_stockM on PERMNO and date
month_features = month_features.merge(monthly_variance_stockM, on = ['PERMNO', 'date'])

# merge temp and market_varianceM on date
month_features = month_features.merge(market_varianceM, on = 'date')

month_features.head()

,PERMNO,date,bull_D,bear_D,bull_W,bear_W,sVarM,mVarM
0,10000,1986-01-31,0,0,1,0,0.004058,0.000068
1,10001,1986-01-31,0,0,0,0,0.000201,0.000068
2,10002,1986-01-31,0,0,0,1,0.000135,0.000068
3,10003,1986-01-31,0,0,1,0,0.000547,0.000068
4,10005,1986-01-31,0,0,0,0,0.002801,0.000068


In [21]:
# misisng values?
month_features.isna().sum()


PERMNO       0
date         0
bull_D       0
bear_D       0
bull_W       0
bear_W       0
sVarM     1712
mVarM        0
dtype: int64

In [97]:
# drop where sVarM is NA
month_features = month_features.dropna(subset = ['sVarM'])


In [98]:
month_features.isna().sum()

PERMNO    0
date      0
bull_D    0
bear_D    0
bull_W    0
bear_W    0
sVarM     0
mVarM     0
dtype: int64

In [99]:
month_features.head()

,PERMNO,date,bull_D,bear_D,bull_W,bear_W,sVarM,mVarM
0,10000,1986-01-31,0,0,1,0,0.004058,0.675148
1,10001,1986-01-31,0,0,0,0,0.000201,0.675148
2,10002,1986-01-31,0,0,0,1,0.000135,0.675148
3,10003,1986-01-31,0,0,1,0,0.000547,0.675148
4,10005,1986-01-31,0,0,0,0,0.002801,0.675148


### Create lagged/cumulative variables and duration, cumulative/lagged market returns will be created later from the CRSP monthly returns data

In [22]:
# sort the data by PERMNO and date
month_features = month_features.sort_values(by = ['PERMNO', 'date']).reset_index(drop = True)

In [23]:
# create a lagged (lag 1 and 2) variable for sVarM and mVarM

month_features['sVarM_lag1'] = month_features.groupby('PERMNO')['sVarM'].shift(1)
month_features['sVarM_lag2'] = month_features.groupby('PERMNO')['sVarM'].shift(2)
month_features['sVarM_lag3'] = month_features.groupby('PERMNO')['sVarM'].shift(3)

month_features['mVarM_lag1'] = month_features.groupby('PERMNO')['mVarM'].shift(1)
month_features['mVarM_lag2'] = month_features.groupby('PERMNO')['mVarM'].shift(2)
month_features['mVarM_lag3'] = month_features.groupby('PERMNO')['mVarM'].shift(3)

In [24]:
month_features.head()

,PERMNO,date,bull_D,bear_D,bull_W,bear_W,sVarM,mVarM,sVarM_lag1,sVarM_lag2,sVarM_lag3,mVarM_lag1,mVarM_lag2,mVarM_lag3
0,10000,1986-01-31,0,0,1,0,0.004058,0.000068,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,1986-02-28,0,0,0,1,0.000973,0.000031,0.004058,NaN,NaN,0.000068,NaN,NaN
2,10000,1986-03-31,0,0,0,0,0.001993,0.000040,0.000973,0.004058,NaN,0.000031,0.000068,NaN
3,10000,1986-04-30,0,1,0,0,0.000163,0.000089,0.001993,0.000973,0.004058,0.000040,0.000031,0.000068
4,10000,1986-05-31,0,0,0,0,0.001569,0.000044,0.000163,0.001993,0.000973,0.000089,0.000040,0.000031


# Save the output: variables coming from daily/weekly data stored on a monthly basis

In [25]:
# save the data to parquet as monthly_features.parquet
month_features.to_parquet("monthly_features.parquet")